In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from numpy.ma.core import exp
import numpy as np
import pandas as pd
from tqdm import tqdm

df = pd.DataFrame()

# 크롤링 데이터를 불러오기 위해 numpy array를 만듭니다.
sale21 = np.array([])
sale20 = np.array([])
ep21 = np.array([])
ep20 = np.array([])
# pr21 = np.array([])
# pr20 = np.array([])

# 2020년, 2021년 opensea 데이터를 가져옵니다.
for i in tqdm(range(1,13)):
  sale20 = np.append(sale20, np.load('./drive/MyDrive/opensea/sales_' + str(i) + '.npy'))
  sale21 = np.append(sale21, np.load('./drive/MyDrive/opensea/22sales_' + str(i) + '.npy'))

  ep20 = np.append(ep20, np.load('./drive/MyDrive/opensea/endprice_' + str(i) + '.npy'))
  ep21 = np.append(ep21, np.load('./drive/MyDrive/opensea/22endprice_' + str(i) + '.npy'))

# df에 데이터를 넣어줍니다.
sale = np.concatenate((sale20, sale21), axis=None)
ep = np.concatenate((ep20, ep21), axis=None)
df['sale'] = sale
df['ep'] = ep

not_null = df['ep'] != 'None'

# # 조건를 충족하는 데이터를 필터링하여 새로운 변수에 저장합니다.
subset_df = df[not_null]
subset_df

In [ ]:
# 판매량, 최종가를 이용하여 비율을 저장합니다.
ratiodf = pd.DataFrame()
ratio = np.array([])
for ind in subset_df.index:
  ratio = np.append(ratio, int(subset_df['ep'][ind]) / (int(subset_df['sale'][ind]) + 1))
ratiodf['value'] = ratio
ratiodf

In [ ]:
# 데이터를 최종적으로 정리합니다.
ratios = np.array(ratiodf['value'])
maginot = np.percentile(ratios, [10,90], interpolation='nearest')
suspect = np.array([])
for ind in ratiodf.index:
  if ratiodf['value'][ind] < maginot[0]:
    suspect = np.append(suspect, 1)
  elif ratiodf['value'][ind] > maginot[1]:
    suspect = np.append(suspect, 1)
  else:
    suspect = np.append(suspect, 0)
subset_df['Class'] = suspect
subset_df['rep'] = rep
subset_df = subset_df.dropna(axis=0)
subset_df = subset_df.reset_index()
subset_df = subset_df[['sale','rep','Class']]
subset_df

In [ ]:
from sklearn.model_selection import train_test_split
# 학습 데이터와 테스트 데이터를 나눠줍니다.
X_train, X_test = train_test_split(subset_df.astype(int), test_size= 0.3, random_state=42) 

# Class를 분리하여줍니다.
y_test = X_test['Class'] 
X_test = X_test.drop(['Class'], axis= 1) 

y_train = X_train['Class'] 
X_train = X_train.drop(['Class'], axis= 1) 

X_train = X_train.values 
X_test = X_test.values 
X_train.shape 

In [ ]:
from sklearn.preprocessing import StandardScaler

# 변형 객체 생성
std_scaler = StandardScaler()

# 훈련데이터의 모수 분포 저장
std_scaler.fit(X_train)

# 훈련 데이터 스케일링
X_train = std_scaler.transform(X_train)

# 테스트 데이터의 스케일링
X_test = std_scaler.transform(X_test)